In [ ]:
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [ ]:
pv_models = {
    "fc_v9": "forecast_v=9__model_name_1=pvnet_app_v__model_version_1=2.3.19__start_date=2019-01-01__end_date=2024-01-01",
    "v5_all": "pvnet_2019-2023_310724_p3_v5_all",
    "fc_v9c": "forecast_v=9c__model_name_1=pvnet_app_v__model_version_1=2.3.19__start_date=2019-01-01__end_date=2024-01-01",
    "fc_v8": "forecast_v=8__model_name_1=national_xg__model_version_1=1.0.23__model_name_2=pv_net__model_version_2=3.0.1__start_date=2016-12-01__end_date=2022-08-08",
}

pvlive_df = pd.read_csv("../data/compare_forecasts/pvlive_2016_2023.csv")

# Load forecast dataframes
forecast_dfs = {}
for short_name, file_name in pv_models.items():
    print(f"Loading {short_name}")
    try:
        df = pd.read_csv(f"../data/compare_forecasts/{file_name}.csv.gz")
    except FileNotFoundError:
        df = pd.read_csv(f"../data/compare_forecasts/{file_name}.csv")
    forecast_dfs[short_name] = df

In [ ]:
forecast_dfs

In [ ]:
# Function to calculate Mean Absolute Error (MAE) for each forecast horizon
def calculate_mae_for_forecast_horizons(
    forecast_dfs, df_actual, min_end_datetime_utc=None, compare_common_init_times=False
):
    mae_results = {}

    if compare_common_init_times:
        # Find common forecasting_creation_datetime_utc across all models
        common_init_times = set.intersection(
            *[set(df["forecasting_creation_datetime_utc"]) for df in forecast_dfs.values()]
        )
        print(f"There are {len(common_init_times)} common init times")

    for model_name, df_forecast in forecast_dfs.items():
        # Merge the forecast and actual dataframes on the end_datetime_utc column
        merged_df = pd.merge(df_forecast, df_actual, on="end_datetime_utc", suffixes=("_forecast", "_actual"))

        # Filter the merged dataframe by the minimum end_datetime_utc if provided
        if min_end_datetime_utc is not None:
            merged_df["end_datetime_utc"] = pd.to_datetime(merged_df["end_datetime_utc"])
            merged_df = merged_df[merged_df["end_datetime_utc"] >= min_end_datetime_utc]

        # Filter for common init times if compare_common_init_times is True
        if compare_common_init_times:
            merged_df = merged_df[merged_df["forecasting_creation_datetime_utc"].isin(common_init_times)]

        # Calculate the difference in hours between the forecasting_creation_datetime_utc and end_datetime_utc
        merged_df["forecast_horizon_hours"] = (
            pd.to_datetime(merged_df["end_datetime_utc"])
            - pd.to_datetime(merged_df["forecasting_creation_datetime_utc"])
        ).dt.total_seconds() / 3600

        # Calculate the absolute error between the forecasted and actual generation
        merged_df["absolute_error"] = np.abs(merged_df["generation_mw_forecast"] - merged_df["generation_mw_actual"])

        # Calculate the error between the forecasted and actual generation
        merged_df["error"] = merged_df["generation_mw_forecast"] - merged_df["generation_mw_actual"]

        # Group by the forecast horizon and calculate the mean absolute error for each group
        mae_by_horizon = merged_df.groupby("forecast_horizon_hours")["absolute_error"].mean().reset_index(name="MAE")
        mbe_by_horizon = merged_df.groupby("forecast_horizon_hours")["error"].mean().reset_index(name="MBE")

        # Merge MAE and MBE results
        results = pd.merge(mae_by_horizon, mbe_by_horizon, on="forecast_horizon_hours")

        mae_results[model_name] = results

    return mae_results

In [ ]:
mae_dfs = calculate_mae_for_forecast_horizons(forecast_dfs, pvlive_df, compare_common_init_times=True)

# Use the min end datetime to calculate the error for just pvnet summation model times
# min_date = pd.to_datetime("2022-01-01 00:00:00").tz_localize("UTC")
# mae_dfs_min = calculate_mae_for_forecast_horizons(forecast_dfs, pvlive_df, min_end_datetime_utc=min_date, compare_common_init_times=False)

In [ ]:
mae_dfs["fc_v9"]

In [ ]:
def calculate_average_mae(mae_dfs):
    # Iterate through each model and calculate the average MAE for the specified time frames
    for model_name, df in mae_dfs.items():
        # Calculate average MAE for 0-8 hours
        avg_mae_0_8 = df[df["forecast_horizon_hours"] <= 8]["MAE"].mean()

        # Calculate average MAE for 0-36 hours
        avg_mae_0_36 = df[df["forecast_horizon_hours"] <= 36]["MAE"].mean()

        # Calculate average MAE for 8-36 hours
        avg_mae_8_36 = df[(df["forecast_horizon_hours"] > 8) & (df["forecast_horizon_hours"] <= 36)]["MAE"].mean()

        # Calculate average MBE for 0-8 hours
        avg_mbe_0_8 = df[df["forecast_horizon_hours"] <= 8]["MBE"].mean()

        # Calculate average MBE for 0-36 hours
        avg_mbe_0_36 = df[df["forecast_horizon_hours"] <= 36]["MBE"].mean()

        # Calculate average MBE for 8-36 hours
        avg_mbe_8_36 = df[(df["forecast_horizon_hours"] > 8) & (df["forecast_horizon_hours"] <= 36)]["MBE"].mean()

        # Print the results
        print(f"Model: {model_name}")
        print(f"Average MAE for 0-8 hours: {avg_mae_0_8:.2f}")
        print(f"Average MAE for 0-36 hours: {avg_mae_0_36:.2f}")
        print(f"Average MAE for 8-36 hours: {avg_mae_8_36:.2f}")
        print(f"Average MBE for 0-8 hours: {avg_mbe_0_8:.2f}")
        print(f"Average MBE for 0-36 hours: {avg_mbe_0_36:.2f}")
        print(f"Average MBE for 8-36 hours: {avg_mbe_8_36:.2f}")
        print()

In [ ]:
calculate_average_mae(mae_dfs)

In [ ]:
def plot_multiple_mae_forecast_horizons(mae_dfs, metric="MAE"):
    df_list = list(mae_dfs.values())
    df_names = list(mae_dfs.keys())

    # Create a plotly figure
    fig = go.Figure()

    # Define colors for the plots
    colors = ["blue", "green", "red", "purple", "orange", "yellow", "brown", "black"]

    for df, name, color in zip(df_list, df_names, colors):
        # Add line plot for MAE or MBE across different forecast horizons for each dataframe
        fig.add_trace(
            go.Scatter(
                x=df["forecast_horizon_hours"], y=df[metric], mode="lines+markers", name=name, line=dict(color=color)
            )
        )

    # Update layout with titles and axis labels
    fig.update_layout(
        title=f"{metric} across Different Forecast Horizons for Multiple Models",
        xaxis_title="Forecast Horizon (hours)",
        yaxis_title=f"{'Mean Absolute Error (MAE)' if metric == 'MAE' else 'Mean Bias Error (MBE)'}",
        template="plotly_white",
    )

    # Show plot
    fig.show()

In [ ]:
plot_multiple_mae_forecast_horizons(mae_dfs)

In [ ]:
plot_multiple_mae_forecast_horizons(mae_dfs, metric="MBE")

In [ ]:
# Function to plot multiple forecasts on the same graph including pvlive_df for 2 days ahead
def plot_multiple_forecasts_with_pvlive(forecasting_datetime, forecast_dfs, pvlive_df):
    fig = go.Figure()

    # Convert forecasting_datetime to datetime object
    forecasting_datetime_obj = datetime.strptime(forecasting_datetime, "%Y-%m-%d %H:%M:%S%z")
    # Calculate 2 days ahead datetime
    two_days_ahead_datetime = forecasting_datetime_obj + timedelta(days=1)

    # Filter pvlive_df for the range
    pvlive_filtered = pvlive_df[
        (pvlive_df["start_datetime_utc"] >= forecasting_datetime)
        & (pvlive_df["end_datetime_utc"] <= two_days_ahead_datetime.isoformat())
    ]

    # Add pvlive data to the plot
    fig.add_trace(
        go.Scatter(
            x=pvlive_filtered["end_datetime_utc"], y=pvlive_filtered["generation_mw"], mode="lines", name="pvlive_data"
        )
    )

    for name, df in forecast_dfs.items():
        pre_gen_data = df[df["forecasting_creation_datetime_utc"] == forecasting_datetime]
        generation_data = pre_gen_data[["end_datetime_utc", "generation_mw"]]

        fig.add_trace(
            go.Scatter(
                x=generation_data["end_datetime_utc"], y=generation_data["generation_mw"], mode="lines", name=name
            )
        )

    fig.update_layout(
        title=f"Generation Data for {forecasting_datetime} including pvlive data for 2 days ahead",
        xaxis_title="End Datetime UTC",
        yaxis_title="Generation MW",
    )
    fig.show()

In [ ]:
# Chose a date to compare
# Find common forecasting_creation_datetime_utc across all models
common_init_times = set.intersection(*[set(df["forecasting_creation_datetime_utc"]) for df in forecast_dfs.values()])
print(f"There are {len(common_init_times)} common init times")

# or pick a date from a model
# Select a specific model from forecast_dfs
model_name = "fc_v9"  # Replace with the actual model name you want to use

if model_name in forecast_dfs:
    # Get unique dates from the selected model
    unique_datetimes = forecast_dfs[model_name]["forecasting_creation_datetime_utc"].unique()
    print(f"There are {len(unique_datetimes)} unique dates for the {model_name} model")


forecasting_datetime = list(common_init_times)[120]  # Convert set to list and get the first element
# forecasting_datetime = "2023-09-02 04:30:00+00:00"
# forecasting_datetime = unique_datetimes[10]

In [ ]:
plot_multiple_forecasts_with_pvlive(forecasting_datetime, forecast_dfs, pvlive_df)